# Differential expression analysis (DEA)

## CpG vs NaCl

In [ ]:
options(warn=-1)
options(dplyr.summarise.inform=FALSE)

In [ ]:
library_load <- suppressMessages(
    
    list(
        
        # Seurat 
        library(Seurat), 
    
        
        # GSEA 
        library(msigdbr), 
        library(fgsea), 
        
        # Data 
        library(tidyverse), 
        library(openxlsx), 
        
        # Plotting 
        library(ComplexHeatmap), 
        library(circlize), 
        library(viridis), 
        library(ggplotify), 
        library(patchwork), 
        library(ggrepel), 
        
        # Python 
        library(reticulate)
        
    )
)

In [ ]:
# Configure reticulate 
use_condaenv(condaenv='p.3.8.12-FD20200109SPLENO', conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
py_config()

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")
source("bin/seurat_qc.R")
source("bin/pbDEA.R")
source("bin/gsea.R")

In [ ]:
# Plotting Theme
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Parameter settings

In [ ]:
# Seurat object file
so_rds_file <- "data/object/pp.rds"

In [ ]:
# DEA settings 
grouping_var="treatment"

random_effect_var="sample_group"
random_effect_var_cell_min=3
random_effect_var_min=2

pseudoreplicate_n=5
pseudoreplicate_var="pseudoreplicate"

cnt_min=3
cell_min=1
feature_group_min=2

p_adj_thr=0.1
sample_weights=FALSE

In [ ]:
# Result basename 
basename <- "result/dea/scRNAseq/nacl_vs_cpg"

In [ ]:
# Use cache 
cache <- FALSE

# Import Seurat object

In [ ]:
so <- readRDS(so_rds_file)

# Check if all cell types have both groups present for DEA

In [ ]:
so$cell_type_fine <- factor(so$cell_type_fine, levels=names(color$cell_type_fine))

In [ ]:
table <- group_by(so@meta.data, cell_type_fine, sample_group) %>% 
    dplyr::summarise(n=n()) %>% 
    reshape2::dcast(., formula=sample_group~cell_type_fine) %>% 
    column_to_rownames(var="sample_group") %>% 
    replace(is.na(.), 0)

knitr::kable(table, "html") %>% as.character() %>% IRdisplay::display_html()

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)

table <- group_by(so@meta.data, cell_type_fine, treatment) %>% 
    dplyr::summarise(n=n()) %>% 
    reshape2::dcast(., formula=treatment~cell_type_fine) %>% 
    column_to_rownames(var="treatment") %>% 
    replace(is.na(.), 0)

ratio <- sweep(table, MARGIN=1, FUN="/", STATS=rowSums(table)) %>% t %>% as.data.frame %>% tibble::rownames_to_column(., "cell_type_fine") %>% 
    dplyr::mutate(label=ifelse(abs(NaCl-CpG)>=0.01, cell_type_fine, NA))

ratio_p <- ggplot(ratio, aes(x=NaCl, y=CpG, color=cell_type_fine, label=label)) + 
    
    geom_point(size=3) + 
    geom_abline() + 
    xlim(0, max(ratio[, 2:3])) + ylim(0, max(ratio[, 2:3])) + 
    geom_text_repel(segment.color="black", force=20, force_pull=1, max.overlaps=getOption("ggrepel.max.overlaps", default=100), size=5, alpha=1, guide="none", segment.size=0.1, color='black') + 
    scale_color_manual(values=color$cell_type_fine) + 
    theme(
        aspect.ratio=1, 
        legend.position="none"
    
    )

ratio_p

# Pseudobatch DEA

In [ ]:
# Set pseudobulk id
so[[pseudoreplicate_var]] <- so@meta.data %>% 
    dplyr::group_by_("cell_type_fine", grouping_var, random_effect_var) %>% 
    dplyr::mutate(pseudoreplicate_id=sample(rep(1:pseudoreplicate_n, length.out=n()))) %>% 
    dplyr::mutate(pseudoreplicate_id=paste0(treatment, "_", sample_rep, "_", pseudoreplicate_id)) %>% 
    dplyr::pull(pseudoreplicate_id)

In [ ]:
# Prepare data 
so <- SplitObject(so, split.by="cell_type_fine")

In [ ]:
# Cell type order 
cell_type_order <- names(color$cell_type_fine)
so <- so[cell_type_order]

In [ ]:
if(!cache) {
    
    # Subset samples 
    so <- lapply(so, group_select, grouping_var=grouping_var, random_effect_var=random_effect_var, random_effect_var_cell_min=random_effect_var_cell_min, random_effect_var_min=random_effect_var_min)
    
    # Subset features
    so <- lapply(so, feature_select, cnt_min=cnt_min, cell_min=cell_min, random_effect_var=random_effect_var, feature_group_min=feature_group_min)
    
    # Voom lmFit
    results <- lapply(so, voomlmfit_group, grouping_var=grouping_var, random_effect_var=random_effect_var, pseudoreplicate_var=pseudoreplicate_var, contrast_vec=c("NaCl", "CpG"), sample_weights=sample_weights) 
    
    # Write xlsx
    write_results(results, basename)
    
} else {
    
    results <- readRDS(paste0(basename, "_dea.rds"))
    
}

In [ ]:
options(repr.plot.width=4*8, repr.plot.height=1*8)

vp_treatment <- lapply(seq_along(results), function(i) {dea_vp(results[[i]], title=names(results)[i], log2fc_thr=0, p_adj_thr=p_adj_thr, color_neg=color$treatment[1], color_pos=color$treatment[2]) + theme_global_set(size_select=1)})
ggpubr::ggarrange(plotlist=vp_treatment, ncol=4, common.legend=FALSE, legend="bottom")

# GSEA

In [ ]:
gs_mm <- msigdbr(species="Mus musculus", category="C2", subcategory="WIKIPATHWAYS")
gs_mm <- split(x=gs_mm$gene_symbol, f=gs_mm$gs_name)

In [ ]:
results <- lapply(results, function(x) {x$gene <- rownames(x); return(x)})

In [ ]:
if(!cache) {
    
    gsea <- lapply(1:length(results), function(i) {gsea_from_dea(dea=results[[i]], gene_set=gs_mm, gene_name="gene")})
    saveRDS(gsea, paste0(basename, "_gsea.rds"))
    
    names(gsea) <- gsub("\\.", "", make.names(names(results)))
    write.xlsx(gsea, paste0(basename, "_gsea.xlsx"), colNames=TRUE)
    
} else {
    
    gsea <- readRDS(paste0(basename, "_gsea.rds"))
    
}

In [ ]:
options(repr.plot.width=2*20, repr.plot.height=1*10)

gsea_p <- lapply(1:length(results), function(i) {gsea_plot(gsea[[i]], pval_thr=p_adj_thr, pathway_suffix="WP", title=names(results)[i]) + theme_global_set(size_select=1)})
ggpubr::ggarrange(plotlist=gsea_p, ncol=2, common.legend=FALSE, legend="bottom")

# Heatmap - Top DEA

In [ ]:
hm_dea_genes <- function(dea_file, cell_type_order, cell_type_color, gene_names, log2fc_sign, padj_thr_group=1, n_slice_group=0, padj_thr=0.1, n_slice=10, color_max="red") {
    
    # DEA results 
    dea <- readRDS(dea_file)
    dea <- do.call(rbind, dea)
    dea <- dea %>% dplyr::filter(ident %in% cell_type_order) 
    
    # Re-set cell types in case some are missing from DEA
    cell_type_order <- cell_type_order[cell_type_order %in% dea$ident]
    
    # Check groups - Don't display group with less DEG than n_sclice for padj_thr_group
    cell_type_group <- dea %>% 
        dplyr::filter(sign(avg_log2FC)==log2fc_sign & p_val_adj <= padj_thr_group) %>% 
        dplyr::group_by(ident) %>% 
        dplyr::summarise(n=n()) %>%
        dplyr::filter(n>=n_slice_group) %>% 
        dplyr::pull(ident) 

    # Get cell type and top DEA genes
    dea_genes <- dea %>%
        dplyr::filter(ident %in% cell_type_group & sign(avg_log2FC)==log2fc_sign) %>% 
        dplyr::group_by(ident) %>% 
        dplyr::arrange(ident, desc(abs(avg_log2FC)), p_val_adj) %>% 
        slice(1:n_slice) %>%
        dplyr::select(ident, gene, p_val_adj) 

    # Get log2FC matrix 
    mat_1 <- dea %>% 
        dplyr::filter(gene %in% dea_genes$gene) %>% 
        dplyr::select(avg_log2FC, ident, gene) %>% 
        reshape2::dcast(., gene ~ ident, value.var="avg_log2FC") %>% 
        replace(is.na(.), 0)
    
    # Get padj matrix 
    mat_2 <- dea %>% 
        dplyr::filter(gene %in% dea_genes$gene) %>% 
        dplyr::select(p_val_adj, ident, gene) %>% 
        reshape2::dcast(., gene ~ ident, value.var="p_val_adj") %>% 
        replace(is.na(.), 1)
    
    # Combine dea_genes with mat and preserve gene names
    mat_1 <- left_join(dea_genes, mat_1, by="gene") %>% 
        dplyr::mutate(ident=factor(ident, levels=cell_type_order)) %>% 
        dplyr::arrange(ident) %>% 
        as.data.frame() 
    
    mat_2 <- left_join(dea_genes, mat_2, by="gene") %>% 
        dplyr::mutate(ident=factor(ident, levels=cell_type_order)) %>% 
        dplyr::arrange(ident) %>% 
        as.data.frame()
    
    # Store column split and column name 
    row_split_1 <- mat_1$ident
    column_names_1 <- mat_1$gene
    
    column_split_2 <- mat_2$ident
    column_names_2 <- mat_2$gene
    
    # Remove cell type and gene names from matirx 
    mat_1 <- mat_1 %>% dplyr::select(-ident, -gene, -p_val_adj) %>% as.matrix 
    mat_2 <- mat_2 %>% dplyr::select(-ident, -gene, -p_val_adj) %>% as.matrix 
    
    # Transform matrix and set column names
    mat_1 <- t(mat_1)
    mat_2 <- t(mat_2)
    
    colnames(mat_1) <- column_names_1
    colnames(mat_2) <- column_names_2
    
    # Color function - expression z-score
    if(log2fc_sign>0) {
        
        color_ramp_mat <- c("#ffffff", max_color=color_max)
        breaks_mat <- seq(0, 2, length.out=length(color_ramp_mat))
    
    } else {
        
        color_ramp_mat <- c(color_max, max_color="#ffffff")
        breaks_mat <- seq(-2,  0, length.out=length(color_ramp_mat))
    
    }

    color_function_mat <- circlize::colorRamp2(breaks_mat, color_ramp_mat) 
    
    # Top annotation - cell type
    bottom_annotation <- HeatmapAnnotation(

        df=data.frame(

            top_annotation_dpi=cell_type_order


        ), 

        annotation_legend_param=list(

            top_annotation_dpi=list(title="Cell type", title_gp=gpar(fontsize=18, fontface="plain"), labels_gp=gpar(fontsize=16), grid_width=unit(6, "mm"), grid_height=unit(6, "mm"))

        ), 

        col=list(

            top_annotation_dpi=cell_type_color

        ), 

        simple_anno_size=unit(6, "mm"), 
        show_annotation_name=FALSE, 
        show_legend=FALSE

    )
    
    # Order columns 
    mat_1 <- mat_1[rev(cell_type_order), ]
    mat_2 <- mat_2[rev(cell_type_order), ]
    
    mat_1 <- t(mat_1)
    mat_2 <- t(mat_2)
    
    # Heat map 
    ComplexHeatmap::Heatmap(

        matrix=mat_1, 
        # name="log2(FC)",  

        col=color_function_mat, 
        na_col="white", 

        width=unit(5.5*ncol(mat_1), "mm"), 
        height=unit(5.5*nrow(mat_1), "mm"), 

        # column_title=NULL, 
        column_title_gp=gpar(fontsize=18, fontface="bold"), 

        row_names_gp=gpar(fontsize=16, fontface="plain"), 
        column_names_gp=gpar(fontsize=18, fontface="plain"), 

        cluster_rows=FALSE, 
        show_row_dend=FALSE,    
        row_split=row_split_1,
        row_title_gp=gpar(fontsize=16, fontface="plain"), 
        row_gap=unit(2, "mm"),
        show_row_names=TRUE,
        row_names_side="right",

        cluster_columns=FALSE,
        cluster_column_slices=FALSE, 
        show_column_dend=FALSE, 
        column_split=NULL,
        column_gap=unit(2, "mm"), 
        show_column_names=TRUE, 
        
        bottom_annotation=bottom_annotation, 
        top_annotation=NULL, 
        right_annotation=NULL, 

        rect_gp=gpar(col=NA, lwd=0, alpha=1), 

        heatmap_legend_param=list(title="log2FC", title_gp=gpar(fontsize=18, fontface="plain"), labels_gp=gpar(fontsize=16), legend_height=unit(5*6, "mm"), grid_width=unit(6, "mm")), 

        border=TRUE, 
        border_gp=gpar(col="black", lwd=unit(0.5, "mm")), 

        use_raster=FALSE, 
        
        cell_fun = function(j, i, x, y, w, h, fill) {
            
        if(mat_2[i, j] < 0.1) {
            
            grid.text("*", x, y)
        
            }
        
        }

    ) 
    
}

In [ ]:
n_slice <- 10

In [ ]:
options(repr.plot.width=15, repr.plot.height=length(results)*n_slice/4.5)

hm_1 <- hm_dea_genes(
    
    dea_file=paste0(basename, "_dea.rds"), 
    cell_type_order=names(color$cell_type_fine), 
    cell_type_color=unlist(color$cell_type_fine),
    gene_names=NULL, 
    log2fc_sign=+1, padj_thr_group=1, n_slice_group=1, padj_thr=padj_thr, n_slice=n_slice, color_max=color$treatment[2]
    
)

ggsave(paste0(basename, "_dea_hm_pos.pdf"), as.ggplot(hm_1), width=15, height=length(results)*n_slice/4.5, dpi=1200, limitsize=FALSE)
hm_1

In [ ]:
options(repr.plot.width=15, repr.plot.height=length(results)*n_slice/4.5)

hm_1 <- hm_dea_genes(
    
    dea_file=paste0(basename, "_dea.rds"), 
    cell_type_order=names(color$cell_type_fine), 
    cell_type_color=unlist(color$cell_type_fine),
    gene_names=NULL, 
    log2fc_sign=-1, padj_thr_group=1, n_slice_group=1, padj_thr=padj_thr, n_slice=n_slice, color_max=color$treatment[1]
    
)

ggsave(paste0(basename, "_dea_hm_neg.pdf"), as.ggplot(hm_1), width=15, height=length(results)*n_slice/4.5, dpi=1200, limitsize=FALSE)
hm_1

# Heatmap - TF 

In [ ]:
hm_dea_genes_bio_class <- function(dea_file, bio_class, column_title=NULL, cell_type_order, cell_type_color, padj_thr=0.1, n_slice=10, color_neg="blue", color_pos="red") {
    
    # DEA results 
    dea <- readRDS(dea_file)
    dea <- do.call(rbind, dea)
    dea <- dea %>% dplyr::filter(ident %in% cell_type_order) 
    
    # Filter for biotype 
    dea <- dea[dea[[bio_class]], ]
    
    # Re-set cell types in case some are missing from DEA
    cell_type_order <- cell_type_order[cell_type_order %in% dea$ident]
    
    # Get cell type and top DEA genes
    dea_genes <- dea %>%
        dplyr::filter(p_val_adj <= padj_thr) %>%
        dplyr::select(ident, gene, p_val_adj)

    # Get log2FC matrix 
    mat_1 <- dea %>% 
        dplyr::filter(gene %in% dea_genes$gene) %>% 
        dplyr::select(avg_log2FC, ident, gene) %>% 
        reshape2::dcast(., gene ~ ident, value.var="avg_log2FC") %>% 
        replace(is.na(.), 0) %>% 
        tibble::column_to_rownames("gene") %>% 
        unique()
    
    # Get padj matrix 
    mat_2 <- dea %>% 
        dplyr::filter(gene %in% dea_genes$gene) %>% 
        dplyr::select(p_val_adj, ident, gene) %>% 
        reshape2::dcast(., gene ~ ident, value.var="p_val_adj") %>% 
        replace(is.na(.), 1) %>% 
        tibble::column_to_rownames("gene") %>% 
        unique()
    
    # Color function - log2FC
    color_ramp_mat <- c(color_neg, "#ffffff", color_pos)
    breaks_mat <- seq(-2,  2, length.out=length(color_ramp_mat))

    color_function_mat <- circlize::colorRamp2(breaks_mat, color_ramp_mat) 
    
    # Top annotation - cell type
    bottom_annotation <- HeatmapAnnotation(

        df=data.frame(

            top_annotation_dpi=cell_type_order


        ), 

        annotation_legend_param=list(

            top_annotation_dpi=list(title="Cell type", title_gp=gpar(fontsize=18, fontface="plain"), labels_gp=gpar(fontsize=16), grid_width=unit(6, "mm"), grid_height=unit(6, "mm"))

        ), 

        col=list(

            top_annotation_dpi=cell_type_color

        ), 

        simple_anno_size=unit(6, "mm"), 
        show_annotation_name=FALSE, 
        show_legend=FALSE

    )    
    
    # Heat map 
    ComplexHeatmap::Heatmap(

        matrix=mat_1, 
        # name="log2(FC)",  

        col=color_function_mat, 
        na_col="white", 

        width=unit(5.5*ncol(mat_1), "mm"), 
        height=unit(5.5*nrow(mat_1), "mm"), 

        column_title=column_title, 
        column_title_gp=gpar(fontsize=18, fontface="bold"), 

        row_names_gp=gpar(fontsize=16, fontface="plain"), 
        column_names_gp=gpar(fontsize=18, fontface="plain"), 

        cluster_rows=TRUE, 
        show_row_dend=FALSE,    
        # row_split=row_split_1,
        row_title_gp=gpar(fontsize=16, fontface="plain"), 
        row_gap=unit(2, "mm"),
        show_row_names=TRUE,
        row_names_side="right",

        cluster_columns=FALSE,
        cluster_column_slices=FALSE, 
        show_column_dend=FALSE, 
        column_split=NULL,
        column_gap=unit(2, "mm"), 
        show_column_names=TRUE, 
        
        bottom_annotation=bottom_annotation, 
        top_annotation=NULL, 
        right_annotation=NULL, 

        rect_gp=gpar(col=NA, lwd=0, alpha=1), 

        heatmap_legend_param=list(title="log2FC", title_gp=gpar(fontsize=18, fontface="plain"), labels_gp=gpar(fontsize=16), legend_height=unit(5*6, "mm"), grid_width=unit(6, "mm")), 

        border=TRUE, 
        border_gp=gpar(col="black", lwd=unit(0.5, "mm")), 

        use_raster=FALSE, 
        
        cell_fun = function(j, i, x, y, w, h, fill) {
            
        if(mat_2[i, j] < 0.1) {
            
            grid.text("*", x, y)
        
            }
        
        }

    )
    
}

In [ ]:
hm_1 <- hm_dea_genes_bio_class(
    
    dea_file=paste0(basename, "_dea.rds"), 
    bio_class="transcription_factor", 
    column_title="TF", 
    cell_type_order=names(color$cell_type_fine), 
    cell_type_color=unlist(color$cell_type_fine),
    padj_thr=p_adj_thr, color_neg=color$treatment[[1]], color_pos=color$treatment[[2]]
    
)

hm_2 <- hm_dea_genes_bio_class(
    
    dea_file=paste0(basename, "_dea.rds"), 
    bio_class="ligand", 
    column_title="Ligand", 
    cell_type_order=names(color$cell_type_fine), 
    cell_type_color=unlist(color$cell_type_fine),
    padj_thr=p_adj_thr, color_neg=color$treatment[[1]], color_pos=color$treatment[[2]]
    
)

hm_3 <- hm_dea_genes_bio_class(
    
    dea_file=paste0(basename, "_dea.rds"), 
    bio_class="receptor", 
    column_title="Receptor", 
    cell_type_order=names(color$cell_type_fine), 
    cell_type_color=unlist(color$cell_type_fine),
    padj_thr=p_adj_thr, color_neg=color$treatment[[1]], color_pos=color$treatment[[2]]
    
)

In [ ]:
options(repr.plot.width=10, repr.plot.height=ceiling(1+dim(hm_1)[1]/4))

ggsave(paste0(basename, "_dea_hm_tf.pdf"), as.ggplot(hm_1), width=15, height=ceiling(1+dim(hm_1)[1]/4), dpi=1200, limitsize=FALSE)
hm_1

In [ ]:
options(repr.plot.width=10, repr.plot.height=ceiling(1+dim(hm_2)[1]/4))

ggsave(paste0(basename, "_dea_hm_ligand.pdf"), as.ggplot(hm_2), width=15, height=ceiling(1+dim(hm_2)[1]/4), dpi=1200, limitsize=FALSE)
hm_2

In [ ]:
options(repr.plot.width=10, repr.plot.height=ceiling(1+dim(hm_3)[1]/4))

ggsave(paste0(basename, "_dea_hm_receptor.pdf"), as.ggplot(hm_3), width=15, height=ceiling(1+dim(hm_3)[1]/4), dpi=1200, limitsize=FALSE)
hm_3

# Compare DEA between cell types

In [ ]:
hm_dea_cor <- function(file, log2FC_sign, score_type, color_min, color_max, padj_thr_gs=0.1, n_thr_gs=5, cache=TRUE) {

    # DEA Ranking and GSEA function 
    gsea_run <- function(dea, gene_set, score_type) {
    
        ranks <- set_names(-log10(dea$p_val_adj) * sign(dea$avg_log2FC), dea$gene)
        ranks <- ranks[order(ranks, decreasing=TRUE)]

        gsea <- fgsea(

            pathways=gene_set,
            stats=ranks,
            nperm=100000, 
            minSize=0, 
            maxSize=Inf, 
            scoreType=score_type

        )

        return(gsea)
    
    }

    # Load DEA results from file 
    dea <- readRDS(paste0(basename, "_dea.rds"))
    dea_order <- names(dea)
    dea <- do.call("rbind", dea)

    # Extract gene sets
    gs_dea <- dplyr::group_by(dea, ident) %>% filter(p_val_adj <= padj_thr_gs & sign(avg_log2FC)==log2FC_sign) %>% dplyr::filter(n() > n_thr_gs)
    gs_dea <- data.frame(gs_name=gs_dea$ident, gene_symbol=gs_dea$gene)
    gs_dea <- split(x=gs_dea$gene_symbol, f=gs_dea$gs_name)

    # Run GSEA 
    dea <- split(dea, f=dea$ident)

    if(!cache) {
        
        gsea <- lapply(dea, function(x) gsea_run(x, gene_set=gs_dea, score_type=score_type))
        saveRDS(gsea, paste0(basename, "_gsea_dea_", score_type, ".rds"))
        
    } else {
        
        gsea <- readRDS(paste0(basename, "_gsea_dea_", score_type, ".rds"))
        
    }
      
    # Get GSEA matrix with NES
    mat <- lapply(gsea, function(x, y) {
    
        x$NES <- ifelse(x$padj <= 0.05, x$NES, 0)
        x <- x[, c("pathway", "NES")]

        order <- y$pathway
        y <- y[!y$pathway %in% x$pathway, , drop=FALSE]

        if(nrow(y) > 0) {x <- rbind(x, y)} else {x <- x}

        x <- tibble::column_to_rownames(x, "pathway") %>% as.data.frame()
        x <- x[order, ]

        return(x)

    }, y=data.frame(pathway=names(gs_dea), NES=NA)

                 )
                   
    mat <- do.call("cbind", mat)
    colnames(mat) <- names(gsea)
    rownames(mat) <- names(gs_dea)
    mat[is.na(mat)] <- 0
    
    # Set geneset to columns 
    mat <- t(mat)
    dea_order <- dea_order[dea_order %in% colnames(mat)]
    mat <- mat[dea_order, dea_order]
                   
    # Get absolut NES values 
    mat <- abs(mat)

    # Color function 
    color_ramp_mat <- c(color_min, color_max)
    breaks_mat <- seq(min(mat, na.rm=TRUE), quantile(mat, 0.99, na.rm=TRUE), length.out=2)
    color_function_mat <- circlize::colorRamp2(breaks_mat, color_ramp_mat) 
                   
    hm <- ComplexHeatmap::Heatmap(
    
        matrix=mat, 

        col=color_function_mat, 
        na_col="white", 

        width=unit(5.5*ncol(mat), "mm"), 
        height=unit(5.5*nrow(mat), "mm"), 

        row_title="DEA", 
        row_title_gp=gpar(fontsize=18, fontface="bold"), 

        column_title="DEA gene module", 
        column_title_gp=gpar(fontsize=18, fontface="bold"), 

        row_names_gp=gpar(fontsize=16, fontface="plain"), 
        column_names_gp=gpar(fontsize=18, fontface="plain"), 

        # cluster_rows=function(mat) hclust(dist(mat), method="complete"), 
        cluster_rows=FALSE, 
        show_row_dend=TRUE,    
        row_gap=unit(2, "mm"),
        show_row_names=TRUE,
        row_names_side="right",

        cluster_columns=FALSE,
        cluster_column_slices=FALSE, 
        show_column_dend=TRUE, 
        column_split=NULL,
        column_gap=unit(2, "mm"), 
        show_column_names=TRUE, 

        top_annotation=NULL, 
        right_annotation=NULL, 

        rect_gp=gpar(col=NA, lwd=0, alpha=1), 

        heatmap_legend_param=list(title="NES", title_gp=gpar(fontsize=18, fontface="plain"), labels_gp=gpar(fontsize=16), legend_height=unit(5*6, "mm"), grid_width=unit(6, "mm")), 

        border=TRUE, 
        border_gp=gpar(col="black", lwd=unit(0.5, "mm")), 

        use_raster=FALSE

    )
                   
    return(hm)
    
}

In [ ]:
hm_1 <- hm_dea_cor(
    
    file=basename, 
    log2FC_sign=+1,
    score_type="pos",
    color_min="#ffffff",
    color_max=color$treatment[2], 
    cache=cache

)

hm_2 <- hm_dea_cor(
    
    file=basename, 
    log2FC_sign=-1,
    score_type="neg",
    color_min="#ffffff",
    color_max=color$treatment[1], 
    cache=cache

)

In [ ]:
options(repr.plot.width=7.5, repr.plot.height=ceiling(2+dim(hm_1)[1]/4))

ggsave(paste0(basename, "_dea_gsea_hm_pos.pdf"), as.ggplot(hm_1), width=15, height=ceiling(2+dim(hm_1)[1]/4), dpi=1200, limitsize=FALSE)
hm_1

In [ ]:
options(repr.plot.width=7.5, repr.plot.height=ceiling(2+dim(hm_2)[1]/4))

ggsave(paste0(basename, "_dea_gsea_hm_neg.pdf"), as.ggplot(hm_2), width=15, height=ceiling(2+dim(hm_2)[1]/4), dpi=1200, limitsize=FALSE)
hm_2